In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sciopy import EIT_16_32_64_128, EitMeasurementSetup
import matplotlib.pyplot as plt

In [ ]:
n_el = 32  #or 64 for 3D measurements
sciospec = EIT_16_32_64_128(n_el)

In [ ]:
# connect device via USB-HS port
sciospec.connect_device_HS()

In [ ]:
# read system message buffer
sciospec.SystemMessageCallback()
# should be empty

In [ ]:
# create a measurement setup
setup = EitMeasurementSetup(
    burst_count=5,
    n_el=n_el,
    exc_freq=125_000,
    framerate=3,
    amplitude=0.01,
    inj_skip=n_el // 2,
    gain=1,
    adc_range=1,
)

In [ ]:
sciospec.SetMeasurementSetup(setup)

In [ ]:
# get data
data = sciospec.StartStopMeasurement(return_as="pot_mat") 
data.shape

In [ ]:
for pot in data:
    plt.imshow(np.abs(pot))
    plt.show()

In [ ]:
import serial

from typing import Union
import time
from datetime import datetime
import numpy as np
import sys
import glob
from src.util import (
    create_trajectory,
    create_trajectory_3D
)
import os
from glob import glob
from tqdm import tqdm
# https://reprap.org/wiki/G-code#M17:_Enable.2FPower_all_stepper_motors

In [ ]:
def command(ser, command) -> None:
    ser.write(str.encode(command))
    time.sleep(1)
    while True:
        line = ser.readline()
        print(line)

        if line == b"ok\n":
            break

# class
class EnderControl:
    def __init__(self,com_port, baudrate = 115200, timeout = 1, motion_speed = 1000):
        self.com_port = com_port
        self.baudrate = baudrate
        self.motion_speed = motion_speed 
        self.position_timestamps = []  

    def connect(self):
        com = serial.Serial(
            port=self.com_port,
            baudrate=self.baudrate,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS,
        )
    
        print("Connection to", com.name, "is established.")
        self.com = com
        
    def init(self):
        """
        Initialise the axis
        """
        # x_y_home(ser)
        command(self.com, f"G28 X0 Y0 F{self.motion_speed}\r\n")
        command(self.com, f"G28 Z0 F{self.motion_speed}\r\n")
        self.x_y_center()
        self.turn_off_fan()
        print("X,Y axis are centered at X(180), Y(180)")

        return [180, 180]
    
    def turn_off_fan(self):
        command(self.com, "M106 S0\r\n")

    def x_y_center(self):
        command(self.com, f"G0 X180 Y180 F{self.motion_speed}\r\n")

    def move_to_absolute_x_y(self, P) -> None:
        command(
            self.com,
            f"G0 X{P[0]+180} Y{P[1]+180} F{self.motion_speed}\r\n",
        )
        pos = [P[0]+180, P[1]+180]
        return pos

    def move_to_absolute_x_y_z(self, P) -> None:
        command(
            self.com,
            f"G0 X{P[0]+180} Y{P[1]+180} Z{P[2]} F{self.motion_speed}\r\n",
        )
        pos = [P[0]+180, P[1]+180, P[2]]  #z im Bereich 50 bis 100 mm
        return pos

    def read_temperature(self) -> float:
        """
        Read the bed temperature of the Ender 5.
    
        This function sends the M105 command, which requests the current temperature.
    
        Returns
        -------
        float
            The bed temperature value.
        """
    
        self.com.write(str.encode(f"M105\r\n"))
        time.sleep(1)
        line = self.com.readline()
        temp = float(str(line).split("B:")[1].split(" ")[0])
        
        return temp

    def get_timestamp(self) -> str:
        """
        Returns the current timestamp in the format YYYY-MM-DD HH:MM:SS
        """
        current_time = datetime.now()  
        formatted_timestamp = current_time.strftime("%Y-%m-%d %H:%M:%S")  
        return current_time, formatted_timestamp

In [ ]:
printer = EnderControl(com_port="/dev/ttyUSB0")
printer.connect()

In [ ]:
printer.init()

In [ ]:
def generate_exp_data_2D(traj, r_path, r_anomaly, material, empty_tank_pos, empty_tank_timestamp, empty_tank_machine_time, empty_tank_temp, empty_tank_data, base_points=1000):

    center_pos = create_trajectory(traj, r_path * 97, base_points)
    base_dataset_folder = "exp_data_set"
    os.makedirs(base_dataset_folder, exist_ok=True)
    
    base_name = "exp_data"
    counter = 1
    folder_name = os.path.join(base_dataset_folder, base_name)
    while os.path.exists(folder_name):
        folder_name = os.path.join(base_dataset_folder, f"{base_name}{counter}")
        counter += 1
    os.makedirs(folder_name)
    
    lookup = {
        "folder": folder_name,
        "Trajectory": traj,
        "Nsteps": base_points,
        "r_anomaly": r_anomaly,
        "r_path": r_path,
        "material": material,
    }

    lookup_file_path = os.path.join(base_dataset_folder, "exp_data_log.txt")
    with open(lookup_file_path, 'a') as f:
        for key, value in lookup.items():
            f.write(f"{key}: {value}\n")
        f.write("-" * 40 + "\n")

    file_path = os.path.join(folder_name, "sample_000000.npz")
    np.savez(file_path, position=empty_tank_pos, timestamp=empty_tank_timestamp, 
             machine_time=empty_tank_machine_time, temperature=empty_tank_temp, v=empty_tank_data)
    
    for save_index, Ps in tqdm(enumerate(center_pos), total=len(center_pos)): 
        pos = printer.move_to_absolute_x_y(Ps)  
        temp = printer.read_temperature()  
        machine_time, timestamp = printer.get_timestamp()  
        data = sciospec.StartStopMeasurement(return_as="pot_mat")  
        
        file_path = os.path.join(folder_name, f"sample_{save_index + 1:06d}.npz")
        np.savez(file_path, position=pos, timestamp=timestamp, machine_time=machine_time, 
                 temperature=temp, v=data)

    print("Measurement is complete!")


def generate_exp_data_3D(traj, r_path, r_anomaly, material, empty_tank_pos, empty_tank_timestamp, empty_tank_machine_time, empty_tank_temp, empty_tank_data, base_points=1000):
   
    fig = plt.figure(figsize=(20, 8))

    n_turns = 20 #2
    
    center_pos = create_trajectory_3D(traj, r_path*97, base_points, n_turns)
    
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.plot(center_pos[:,0], center_pos[:,1], center_pos[:,2], color='red', label='Trajectory')
    ax1.scatter(center_pos[:, 0], center_pos[:, 1], center_pos[:, 2], color='blue', marker='o', s=1, label='Points')
    ax1.view_init(elev=90, azim=0)  # top view
    ax1.set_title('Top View')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    
    ax2 = fig.add_subplot(132, projection='3d')
    ax2.plot(center_pos[:,0], center_pos[:,1], center_pos[:,2], color='red', label='Trajectory')
    ax2.scatter(center_pos[:, 0], center_pos[:, 1], center_pos[:, 2], color='blue', marker='o', s=1, label='Points')
    ax2.view_init(elev=0, azim=0)  # side view
    ax2.set_title('Side View')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    
    ax3 = fig.add_subplot(133, projection='3d')
    ax3.plot(center_pos[:,0], center_pos[:,1], center_pos[:,2], color='red', label='Trajectory')
    ax3.scatter(center_pos[:, 0], center_pos[:, 1], center_pos[:, 2], color='blue', marker='o', s=1, label='Points')
    ax3.view_init(elev=45, azim=45)  # isometric view
    ax3.set_title('Isometric View')
    ax3.set_xlabel('X')
    ax3.set_ylabel('Y')
    ax3.set_zlabel('Z')

    plt.tight_layout()
    for ax in [ax1, ax2, ax3]:
        ax.legend()
    
    plt.show()

    base_dataset_folder = "exp_data_set_3D"
    os.makedirs(base_dataset_folder, exist_ok=True)
    
    base_name = "exp_data_3D"
    counter = 1
    folder_name = os.path.join(base_dataset_folder, base_name)
    while os.path.exists(folder_name):
        folder_name = os.path.join(base_dataset_folder, f"{base_name}{counter}")
        counter += 1
    os.makedirs(folder_name)
    
    lookup = {
        "folder": folder_name,
        "Trajectory": traj,
        "Nsteps": base_points,
        "r_anomaly": r_anomaly,
        "r_path": r_path,
        "material": material,
    }

    lookup_file_path = os.path.join(base_dataset_folder, "exp_data_3D_log.txt")
    with open(lookup_file_path, 'a') as f:
        for key, value in lookup.items():
            f.write(f"{key}: {value}\n")
        f.write("-" * 40 + "\n")

    file_path = os.path.join(folder_name, "sample_000000.npz")
    empty_tank_pos = [180, 180, 00]
    np.savez(file_path, position=empty_tank_pos, timestamp=empty_tank_timestamp, 
             machine_time=empty_tank_machine_time, temperature=empty_tank_temp, v=empty_tank_data)
    
    for save_index, Ps in tqdm(enumerate(center_pos), total=len(center_pos)): 
        pos = printer.move_to_absolute_x_y_z(Ps)  
        temp = printer.read_temperature()  
        machine_time, timestamp = printer.get_timestamp()  
        data = sciospec.StartStopMeasurement(return_as="pot_mat")  
        
        file_path = os.path.join(folder_name, f"sample_{save_index + 1:06d}.npz") 
        np.savez(file_path, position=pos, timestamp=timestamp, machine_time=machine_time, 
                 temperature=temp, v=data)

    print("Measurement is complete!")


def empty_tank_measure():
    pos = [180, 180]
    temp = printer.read_temperature()
    machine_time, timestamp = printer.get_timestamp()
    data = sciospec.StartStopMeasurement(return_as="pot_mat")
    return pos, timestamp, machine_time, temp, data

In [ ]:
empty_tank_pos, empty_tank_timestamp, empty_tank_machine_time, empty_tank_temp, empty_tank_data = empty_tank_measure()

In [ ]:
generate_exp_data_2D(
    "Quadrat",          
    0.5,              
    20,               
    "Acryl Glas",     
    empty_tank_pos,  
    empty_tank_timestamp,  
    empty_tank_machine_time,  
    empty_tank_temp,  
    empty_tank_data
)